In [1]:
# Importing relevant modules
import pandas as pd
import panel as pn
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import matplotlib_inline
import hvplot.pandas

# Initializing panel tabulator
pn.extension('tabulator')

# STEP 1: UNDERSTANDING THE PROVIDED DATA
> Here, we are going to try and inspect the data to find out:
> 1. The Number of Columns
> 2. The Number of Rows
> 3. The Data Types on each column
> 4. Columns names etc.

### Let's read the data file

In [2]:
raw_data = pd.read_csv("Hospital ER.csv")

### Viewing a snippet of the data

In [3]:
raw_data

,date,patient_id,patient_gender,patient_age,patient_sat_score,patient_first_inital,patient_last_name,patient_race,patient_admin_flag,patient_waittime,department_referral
0,2020-03-20 08:47:01,145-39-5406,M,69,10.0,H,Glasspool,White,False,39,NaN
1,2020-06-15 11:29:36,316-34-3057,M,4,NaN,X,Methuen,Native American/Alaska Native,True,27,NaN
2,2020-06-20 09:13:13,897-46-3852,F,56,9.0,P,Schubuser,African American,True,55,General Practice
3,2020-02-04 22:34:29,358-31-9711,F,24,8.0,U,Titcombe,Native American/Alaska Native,True,31,General Practice
4,2020-09-04 17:48:27,289-26-0537,M,5,NaN,Y,Gionettitti,African American,False,10,Orthopedics
...,...,...,...,...,...,...,...,...,...,...,...
9211,2020-10-25 02:35:03,453-45-2632,M,60,NaN,A,Crich,African American,False,34,NaN
9212,2019-09-29 15:50:17,136-93-5822,M,63,NaN,A,Gerbl,White,False,55,NaN
9213,2020-08-12 14:46:37,822-41-7734,F,27,NaN,J,Dowall,Two or More Races,False,52,Orthopedics
9214,2019-10-11 16:46:15,260-14-4032,M,53,NaN,W,Rustan,White,True,51,NaN


### Let's use describe to get more insights on the data

In [4]:
raw_data.describe()

,patient_age,patient_sat_score,patient_waittime
count,9216.000000,2517.000000,9216.000000
mean,39.855143,4.992054,35.259874
std,22.755125,3.138043,14.735323
min,1.000000,0.000000,10.000000
25%,20.000000,2.000000,23.000000
50%,39.000000,5.000000,35.000000
75%,60.000000,8.000000,48.000000
max,79.000000,10.000000,60.000000


### Checking the datatypes of the various columns

In [5]:
raw_data.dtypes

date                     object
patient_id               object
patient_gender           object
patient_age               int64
patient_sat_score       float64
patient_first_inital     object
patient_last_name        object
patient_race             object
patient_admin_flag         bool
patient_waittime          int64
department_referral      object
dtype: object

### Checking if there are duplicated records

In [6]:
duplicates = raw_data.duplicated()

# Generate a list of duplicate records
duplicate_records = [raw_data.iloc[index,:] for index, record in enumerate(duplicates) if record == True]

# Patients the duplicate records
# number_of_duplicates = duplicates.where(duplicates==True).count()

# Output the count
# number_of_duplicates
duplicate_records

[]

### From the above snippets we can tell that:
> 1. We have 9216 rows and 11 columns
> 2. Out of the 11 columns, we have 2 int64, 1 float, 1 boolean and the rest are objects
> 3. We have zero duplicates which is a good thing
> 4. We have null values in some of the columns and so we will need to clean the data further.

# STEP 2: DATA CLEANING
> As part of the cleaning exercise, we are going to remove the NaNs and replace them with custom values 

### We will first need to make a copy of the raw data. The changes will only be done on the copy

In [7]:
new_data = raw_data

### The first column that we are going to work on is the department_referal column
> We aren't gonna touch the patient_sat_score because the NaN implies the patient never gave a review for whatever reason

In [8]:
new_data.fillna({'department_referral': 'Unspecified'}, inplace=True)
new_data.head()

,date,patient_id,patient_gender,patient_age,patient_sat_score,patient_first_inital,patient_last_name,patient_race,patient_admin_flag,patient_waittime,department_referral
0,2020-03-20 08:47:01,145-39-5406,M,69,10.0,H,Glasspool,White,False,39,Unspecified
1,2020-06-15 11:29:36,316-34-3057,M,4,NaN,X,Methuen,Native American/Alaska Native,True,27,Unspecified
2,2020-06-20 09:13:13,897-46-3852,F,56,9.0,P,Schubuser,African American,True,55,General Practice
3,2020-02-04 22:34:29,358-31-9711,F,24,8.0,U,Titcombe,Native American/Alaska Native,True,31,General Practice
4,2020-09-04 17:48:27,289-26-0537,M,5,NaN,Y,Gionettitti,African American,False,10,Orthopedics


## We will also change the data type for the date column to datetime

In [9]:
dates_column = new_data['date']

new_data = new_data.assign(date=pd.to_datetime(dates_column))
new_data.dtypes

date                    datetime64[ns]
patient_id                      object
patient_gender                  object
patient_age                      int64
patient_sat_score              float64
patient_first_inital            object
patient_last_name               object
patient_race                    object
patient_admin_flag                bool
patient_waittime                 int64
department_referral             object
dtype: object

# STEP 3: PROCESSING THE DATA
> Some of the insights that we might want to present with the data include:
> 1. The gender distribution of the patients; We will use a pie chart/donut chart to present this insight
> 2. The age groups; A bar or line graph
> 3. Their race; A horizontal bar graph
> 4. The patient satisfaction rate; A metre
> 5. Average patient satisfaction rate per department
> 6. Average patient wait time: 
> 7. The department referals rate: A horizontal bar graph
> 8. Times in the day with the most activity:
> 9. Readmission rates
> 10. Hospital Patient admission rate
> 11. Departmental Patient admission rate

### The frame that we will be working with

In [10]:
Data = new_data.assign(Patients=1)
Data.head()

,date,patient_id,patient_gender,patient_age,patient_sat_score,patient_first_inital,patient_last_name,patient_race,patient_admin_flag,patient_waittime,department_referral,Patients
0,2020-03-20 08:47:01,145-39-5406,M,69,10.0,H,Glasspool,White,False,39,Unspecified,1
1,2020-06-15 11:29:36,316-34-3057,M,4,NaN,X,Methuen,Native American/Alaska Native,True,27,Unspecified,1
2,2020-06-20 09:13:13,897-46-3852,F,56,9.0,P,Schubuser,African American,True,55,General Practice,1
3,2020-02-04 22:34:29,358-31-9711,F,24,8.0,U,Titcombe,Native American/Alaska Native,True,31,General Practice,1
4,2020-09-04 17:48:27,289-26-0537,M,5,NaN,Y,Gionettitti,African American,False,10,Orthopedics,1


### Insight 1: Patients Population by gender

In [11]:
Patients_Gender = Data[['patient_gender', 'Patients']].groupby('patient_gender', as_index=False).sum()
# Let's Find the Percentage of each gender
"""
    We take the sum of the count column and divide it by the value of each row then multiply by 100 and round off to 2 decimal places
"""
Gender_Distribution = Patients_Gender.assign(Percentage=lambda x: [round(x/Patients_Gender['Patients'].sum()*100, 2) for x in Patients_Gender['Patients']])
Gender_Distribution

,patient_gender,Patients,Percentage
0,F,4487,48.69
1,M,4705,51.05
2,NC,24,0.26


### Insight 2: Patients Population by age
> We will have 5 age categories; Children, teens, Young Adults, Middle Aged Adults, Senior Adults
> 1. age <= 12 -> Children
> 2. age >= 13 && <= 19 -> teens
> 3. age >= 20 && <= 35 -> Young Adults
> 4. age >= 36 && <= 65 -> Middle aged Adults
> 5. age > 65 Senior Adults

In [12]:
# Let's first get the patient age and count columns
Age_df = Data[['patient_age', 'Patients']]

# Grouping the Various Ages into proper categories
Age_df2 = Age_df.assign(patient_category=lambda age: 
                        [
                            (
                                "Children" if age <= 12 else 
                                "Teens" if age >=13 and age <= 19 else 
                                "Young Adults" if age >= 20 and age <= 35 else 
                                "Middle Aged Adults" if age >= 36 and age <= 65 else 
                                "Senior Adults"
                            ) for age in Age_df['patient_age']
                        ]
                       )
Age_Distribution = Age_df2[['patient_category', 'Patients']].groupby('patient_category', as_index=False).sum()
Number_of_Patients = Age_Distribution['Patients']
Age_Distribution = Age_Distribution.assign(Percentage=lambda percent: [round(x/Number_of_Patients.sum()*100,1) for x in Number_of_Patients])
Age_Distribution

,patient_category,Patients,Percentage
0,Children,1413,15.3
1,Middle Aged Adults,3504,38.0
2,Senior Adults,1617,17.5
3,Teens,822,8.9
4,Young Adults,1860,20.2


### Insight 3: Patients Population by Race

In [13]:
race_df = Data[['patient_race', 'Patients']].groupby('patient_race', as_index=False).sum()
race_population = race_df['Patients']
race_Distribution = race_df.assign(Percentage=lambda x: [round(x/race_population.sum()*100, 1) for x in race_population])
race_Distribution

,patient_race,Patients,Percentage
0,African American,1951,21.2
1,Asian,1060,11.5
2,Declined to Identify,1030,11.2
3,Native American/Alaska Native,498,5.4
4,Pacific Islander,549,6.0
5,Two or More Races,1557,16.9
6,White,2571,27.9


### Insight 4: Departmental Referrals

In [14]:
department_df = Data[['department_referral','Patients']].groupby('department_referral', as_index=False).sum()
department_df['Percentage'] = round(department_df['Patients']/department_df['Patients'].sum() * 100, 1)
department_df

,department_referral,Patients,Percentage
0,Cardiology,248,2.7
1,Gastroenterology,178,1.9
2,General Practice,1840,20.0
3,Neurology,193,2.1
4,Orthopedics,995,10.8
5,Physiotherapy,276,3.0
6,Renal,86,0.9
7,Unspecified,5400,58.6


### Insight 5: Hospital Admission Rates

In [15]:
admission_df = Data[['patient_admin_flag', 'Patients']].groupby('patient_admin_flag', as_index=False).sum()
admission_df = admission_df.assign(Percentage=lambda x: [round(x/admission_df['Patients'].sum()*100, 2) for x in admission_df['Patients']])
admission_Distribution = admission_df.T.rename(columns={0:'out_patients', 1: 'in_patients'}).iloc[1:,:]
admission_Distribution

,out_patients,in_patients
Patients,4604,4612
Percentage,49.96,50.04


### Insight 6: Department Admission Rates

In [16]:
Department_Admin_Rate = Data[['department_referral', 'patient_admin_flag', 'Patients']].groupby(['department_referral', 'patient_admin_flag'], as_index=False).sum()
Dep_Admissions = Department_Admin_Rate['Patients'].where(Department_Admin_Rate.patient_admin_flag==True)
Dep_Rates = pd.DataFrame(columns=['department_referral', 'out_patients', 'in_patients'])
Dep_Rates['department_referral'] = Department_Admin_Rate['department_referral'].unique()
Dep_Rates['in_patients'] = Department_Admin_Rate['Patients'].where(Department_Admin_Rate.patient_admin_flag==True).dropna().reset_index().drop(columns=['index'])
Dep_Rates['out_patients'] = Department_Admin_Rate['Patients'].where(Department_Admin_Rate.patient_admin_flag==False).dropna().reset_index().drop(columns=['index'])
Department_Admin_Distrib = Dep_Rates.assign(Adm_Percentage=round(Dep_Rates.in_patients/(Dep_Rates['out_patients']+Dep_Rates['in_patients'])*100, 1))
Department_Admin_Distrib

,department_referral,out_patients,in_patients,Adm_Percentage
0,Cardiology,126.0,122.0,49.2
1,Gastroenterology,89.0,89.0,50.0
2,General Practice,953.0,887.0,48.2
3,Neurology,96.0,97.0,50.3
4,Orthopedics,497.0,498.0,50.1
5,Physiotherapy,139.0,137.0,49.6
6,Renal,40.0,46.0,53.5
7,Unspecified,2664.0,2736.0,50.7


### Insight 7: Patient Satisfaction Rate
> Keep in mind that there are NaNs to mean that that particular patient didn't give their ratings

In [17]:
satisfaction_df = Data[['Patients', 'patient_sat_score']].dropna()
satisfaction_score = satisfaction_df.groupby('patient_sat_score', as_index=False).sum()
satisfaction_score['Percentage'] = round(satisfaction_score['Patients']/satisfaction_score['Patients'].sum()*100, 1)
satisfaction_score

,patient_sat_score,Patients,Percentage
0,0.0,222,8.8
1,1.0,246,9.8
2,2.0,204,8.1
3,3.0,228,9.1
4,4.0,248,9.9
5,5.0,221,8.8
6,6.0,231,9.2
7,7.0,256,10.2
8,8.0,218,8.7
9,9.0,222,8.8


> Getting overall average satisfaction rate

In [18]:
score_product = satisfaction_score['patient_sat_score'] * satisfaction_score['Patients']
score_sum = score_product.sum()
no_of_patients = satisfaction_score['Patients'].sum()
average_score = round(score_sum/no_of_patients, 1)
average_score

5.0

### Insight 8: Department Based Satisfaction Rates

> 1. #### Get the individual satsfaction score for the departments

In [19]:
department_sat_df = Data[['department_referral', 'patient_sat_score', 'Patients']].dropna()
department_sat_score = department_sat_df.groupby(['department_referral', 'patient_sat_score'], as_index=False).sum()
department_sat_score = department_sat_score.pivot_table(index='patient_sat_score', columns='department_referral', values='Patients').astype('int')
department_sat_score.columns.name = None
department_sat_score.index.name = 'Score'
total_ratees = department_sat_score.sum(axis=0)
department_sat_rate = round(department_sat_score.div(total_ratees, axis=1) * 100, 1)

department_sat_rate

,Cardiology,Gastroenterology,General Practice,Neurology,Orthopedics,Physiotherapy,Renal,Unspecified
Score,,,,,,,,
0.0,5.6,1.9,9.1,5.7,11.4,10.8,13.0,8.5
1.0,14.1,13.0,9.3,17.0,9.0,3.6,4.3,9.9
2.0,4.2,3.7,7.4,9.4,9.0,13.3,8.7,8.2
3.0,7.0,5.6,8.9,3.8,10.7,3.6,4.3,9.6
4.0,15.5,5.6,7.4,5.7,7.6,13.3,26.1,10.8
5.0,2.8,13.0,11.3,9.4,8.3,13.3,4.3,7.9
6.0,9.9,13.0,10.1,9.4,7.6,9.6,13.0,8.9
7.0,18.3,11.1,9.3,1.9,10.0,3.6,4.3,10.8
8.0,5.6,13.0,8.9,9.4,8.3,10.8,13.0,8.4


> 2. #### Get the Average Satisfaction for the departments

In [20]:
dep_sat_score = department_sat_df.groupby('department_referral', as_index=False).sum()
"""
    We are going to divide the sum satisfaction by the Patients 
    multiplied by 10(maximum score) then divide the result by 10. 
    Since the x10 and div 10 cancel each other we will ignore the multiply and division operation leaving us with
    Average_Patient_Sat_Score = patient_sat_score/Patients
"""
dep_sat_score['patient_sat_score'] = round(dep_sat_score['patient_sat_score']/dep_sat_score['Patients'], 2)
dep_sat_score.rename(columns={'patient_sat_score': 'avg_sat_score'}, inplace=True)
dep_sat_score.drop(columns='Patients', inplace=True)
dep_sat_score

,department_referral,avg_sat_score
0,Cardiology,5.14
1,Gastroenterology,5.80
2,General Practice,5.06
3,Neurology,5.28
4,Orthopedics,4.86
5,Physiotherapy,4.99
6,Renal,4.57
7,Unspecified,4.95


### Insight 9: ER Traffic on various times of the day and days of the week

> 1. #### First is categorizing per hour

Extract the day of the week and time of the day

In [21]:
Traffic_df = Data[['date', 'Patients']]
Hourly_Traffic_df = Traffic_df.assign(day_of_week=Traffic_df['date'].dt.strftime('%a'), time_of_day=Traffic_df['date'].dt.hour)
Hourly_Traffic_df.head()

,date,Patients,day_of_week,time_of_day
0,2020-03-20 08:47:01,1,Fri,8
1,2020-06-15 11:29:36,1,Mon,11
2,2020-06-20 09:13:13,1,Sat,9
3,2020-02-04 22:34:29,1,Tue,22
4,2020-09-04 17:48:27,1,Fri,17


In [22]:
Hourly_pop_in_er = Hourly_Traffic_df[["day_of_week", 'time_of_day', 'Patients']]

unit_milliseconds = 3600000

# Get the number of patients
Hourly_pop_in_er_grouped = Hourly_pop_in_er.groupby(['day_of_week', 'time_of_day'], as_index=False).count()

Hourly_pop_in_er_pivot_df = Hourly_pop_in_er_grouped.pivot_table(index='time_of_day', columns='day_of_week', values='Patients', aggfunc='first')

# Reset column names
Hourly_pop_in_er_pivot_df.columns.name = None

# Reset index name
Hourly_pop_in_er_pivot_df.index.name = 'Hours'
Hourly_pop_in_er_pivot_df = Hourly_pop_in_er_pivot_df[['Sun','Mon','Tue','Wed','Thu','Fri','Sat']]
# population_in_er_pivot_df = population_in_er_pivot_df.reindex(['Sun','Mon','Tue','Wed','Thu','Fri','Sat'])

# Calculate the sum of population for each day
daily_sums = Hourly_pop_in_er_pivot_df.sum(axis=1)

# Divide each value in the DataFrame by its corresponding daily sum and multiply by 100
df_Hourly_percentage = round(Hourly_pop_in_er_pivot_df.div(daily_sums, axis=0) * 100, 1)
df_Hourly_percentage.head()
# datetime.hour.
# df_Hourly_percentage.index.values

,Sun,Mon,Tue,Wed,Thu,Fri,Sat
Hours,,,,,,,
0,14.0,14.3,13.3,16.3,14.3,13.3,14.5
1,15.1,16.1,17.2,12.9,13.2,12.9,12.6
2,15.4,16.8,13.6,13.0,14.1,14.9,12.2
3,14.3,16.9,12.2,16.1,14.0,12.5,14.0
4,10.2,14.1,14.8,18.0,15.4,12.8,14.8


> 2. #### Categorizing in terms of morning, afternoon and evening

In [23]:
end_of_morning = 11
end_of_afternoon = 17

def assign_time(timeOfDay):
    if timeOfDay >= end_of_afternoon:
        return "Evening"
    elif timeOfDay >= end_of_morning:
        return "Afternoon"
    else:
        return "Morning"

Timely_Traffic = Traffic_df.assign(day_of_week=Traffic_df['date'].dt.strftime('%a'), time_of_day=Hourly_Traffic_df['time_of_day'].apply(assign_time))

Timely_Traffic.head()

,date,Patients,day_of_week,time_of_day
0,2020-03-20 08:47:01,1,Fri,Morning
1,2020-06-15 11:29:36,1,Mon,Afternoon
2,2020-06-20 09:13:13,1,Sat,Morning
3,2020-02-04 22:34:29,1,Tue,Evening
4,2020-09-04 17:48:27,1,Fri,Evening


> #### Then piecing it in a more presentable format
> NB: The populations are represented in percentages

In [24]:
population_in_er = Timely_Traffic[["day_of_week", 'time_of_day', 'Patients']]

# Get the number of patients
population_in_er_grouped = population_in_er.groupby(['day_of_week', 'time_of_day'], as_index=False).count()

population_in_er_pivot_df = population_in_er_grouped.pivot_table(index='day_of_week', columns='time_of_day', values='Patients', aggfunc='first')

# Reset column names
population_in_er_pivot_df.columns.name = None

# Reset index name
population_in_er_pivot_df.index.name = 'days'
population_in_er_pivot_df = population_in_er_pivot_df[['Morning', 'Afternoon', 'Evening']]
population_in_er_pivot_df = population_in_er_pivot_df.reindex(['Sun','Mon','Tue','Wed','Thu','Fri','Sat'])

# Calculate the sum of population for each day
daily_sums = population_in_er_pivot_df.sum(axis=1)

# Divide each value in the DataFrame by its corresponding daily sum and multiply by 100
df_percentage = round(population_in_er_pivot_df.div(daily_sums, axis=0) * 100, 1)
df_percentage


,Morning,Afternoon,Evening
days,,,
Sun,46.0,25.7,28.3
Mon,45.9,25.3,28.8
Tue,45.4,24.7,29.8
Wed,46.3,23.7,30.0
Thu,47.4,24.8,27.8
Fri,45.2,26.9,27.9
Sat,45.0,25.1,29.9


> Displaying departmental populations over different days

In [25]:
pop_head = Data[['date', 'department_referral']]
# population_in_er_dep = Timely_Traffic[["day_of_week", 'time_of_day', 'Patients']]

population_in_er_dep = pop_head.merge(Timely_Traffic, on="date")[["department_referral", "day_of_week", 'Patients']]

# Get the number of patients
population_in_er_dep_grouped = population_in_er_dep.groupby(['department_referral', 'day_of_week'], as_index=False).count()

population_in_er_dep_pivot_df = population_in_er_dep_grouped.pivot_table(index='department_referral', columns='day_of_week', values='Patients', aggfunc='first')

# Reset column names
population_in_er_dep_pivot_df.columns.name = None

# Reset index name
population_in_er_dep_pivot_df.index.name = 'departments'
population_in_er_dep_pivot_df = population_in_er_dep_pivot_df[['Sun','Mon','Tue','Wed','Thu','Fri','Sat']]

# Calculate the sum of population for each day
daily_dep_sums = population_in_er_dep_pivot_df.sum(axis=1)

# Divide each value in the DataFrame by its corresponding daily sum and multiply by 100
df_dep_percentage = round(population_in_er_dep_pivot_df.div(daily_dep_sums, axis=0) * 100, 1)

df_dep_percentage

,Sun,Mon,Tue,Wed,Thu,Fri,Sat
departments,,,,,,,
Cardiology,15.7,16.1,11.7,14.9,12.5,12.1,16.9
Gastroenterology,14.0,15.7,14.6,14.6,12.9,11.2,16.9
General Practice,13.1,15.1,13.0,15.2,16.1,13.0,14.5
Neurology,18.7,13.0,13.0,17.6,13.5,14.5,9.8
Orthopedics,14.0,14.7,14.1,14.3,12.5,14.6,16.0
Physiotherapy,13.0,18.1,13.0,15.2,12.7,15.2,12.7
Renal,8.1,11.6,14.0,14.0,23.3,22.1,7.0
Unspecified,14.6,14.8,15.0,13.7,13.9,13.6,14.3


### Insight 10: Patient Waiting Time
> 1. Average Overall waiting time
> 2. Average Overall waiting time in different times of the day
> 3. Waiting time per department

> 1. #### Overall average waiting time

In [26]:
# The time is recorded in minutes
waiting_time = Data[['patient_waittime']]

avg_waiting_time = round(waiting_time.sum()/waiting_time.count(), 0).astype('int')
avg_waiting_time

patient_waittime    35
dtype: int64

> 2. #### Overall average waiting time in different times of the day

In [27]:
waiting_per_time_df = Data[['date', 'patient_waittime']]

# Merging the Timely Traffic we go earlier to the new patient_wait df
waiting_per_time_df = waiting_per_time_df.merge(Timely_Traffic[['date', 'time_of_day']], on='date')

# Grouping to get the sum minutes patients have been waiting in each time packet and also counting the patients in each packet
Overall_Waiting = waiting_per_time_df[['time_of_day', 'patient_waittime']].groupby('time_of_day', as_index=False).sum()
count_per_category = waiting_per_time_df[['time_of_day', 'patient_waittime']].groupby('time_of_day', as_index=False).count()

Overall_Waiting['avg_waittime'] = Overall_Waiting['patient_waittime'].apply(lambda x: int(round([x/count for count in count_per_category['patient_waittime']][0], 0)))
Avg_Ovrl_Waittime = Overall_Waiting[['time_of_day', 'avg_waittime']]
Avg_Ovrl_Waittime = Avg_Ovrl_Waittime.merge(count_per_category, on='time_of_day')
Avg_Ovrl_Waittime.rename(columns={'patient_waittime': 'No_of_patients'}, inplace=True)
Avg_Ovrl_Waittime

,time_of_day,avg_waittime,No_of_patients
0,Afternoon,35,2319
1,Evening,41,2668
2,Morning,65,4229


> 3. #### Average waiting time per department

In [28]:
dep_waiting_time_df = Data[['department_referral', 'patient_waittime']]
dep_waittime = dep_waiting_time_df.groupby('department_referral', as_index=False).sum()
dep_patient_count = dep_waiting_time_df.groupby('department_referral', as_index=False).count()

dep_waittime['avg_waittime'] = dep_waittime['patient_waittime'].apply(lambda x: int(round([x/count for count in dep_patient_count['patient_waittime']][0], 0)))
Avg_dep_waiting_time = dep_waittime[['department_referral', 'avg_waittime']]
Avg_dep_waiting_time = Avg_dep_waiting_time.merge(dep_patient_count, on='department_referral')
Avg_dep_waiting_time.rename(columns={'patient_waittime': 'No_of_patients'}, inplace=True)
Avg_dep_waiting_time

,department_referral,avg_waittime,No_of_patients
0,Cardiology,35,248
1,Gastroenterology,26,178
2,General Practice,259,1840
3,Neurology,29,193
4,Orthopedics,140,995
5,Physiotherapy,41,276
6,Renal,12,86
7,Unspecified,768,5400


> 4. Average waiting time per department in different times of the day

In [29]:
Avg_dep_time = Data[['date', 'department_referral', 'patient_waittime']]
depandtime_wait_df = Avg_dep_time.merge(Timely_Traffic[['date', 'time_of_day']], on='date')

# Get the total time the patients waited in each category
temp_time_data = depandtime_wait_df[['department_referral', 'patient_waittime', 'time_of_day']].groupby(['department_referral', 'time_of_day'], as_index=False).sum()

# Get the number of patients
temp_patient_pop = depandtime_wait_df[['department_referral', 'patient_waittime', 'time_of_day']].groupby(['department_referral', 'time_of_day'], as_index=False).count()

temp_data = temp_time_data

temp_data['patient_waittime'] = round(temp_time_data['patient_waittime']/temp_patient_pop['patient_waittime'], 0).astype('int')
dep_pivot_df = temp_data.pivot_table(index='department_referral', columns='time_of_day', values='patient_waittime', aggfunc='first')

# Reset column names
dep_pivot_df.columns.name = None

# Reset index name
dep_pivot_df.index.name = 'department'
dep_pivot_df = dep_pivot_df[['Morning', 'Afternoon', 'Evening']]
dep_pivot_df

,Morning,Afternoon,Evening
department,,,
Cardiology,36,34,35
Gastroenterology,36,37,34
General Practice,35,35,35
Neurology,36,38,37
Orthopedics,36,34,34
Physiotherapy,37,37,35
Renal,34,36,35
Unspecified,35,35,36


> 5. Average waiting time per department in different days of the week

In [30]:
Avg_daily_dep_time = Data[['date', 'department_referral', 'patient_waittime']]
dailydepandtime_wait_df = Avg_dep_time.merge(Timely_Traffic[['date', 'day_of_week']], on='date')

# Get the total time the patients waited in each category
daily_temp_time_data = dailydepandtime_wait_df[['department_referral', 'patient_waittime', 'day_of_week']].groupby(['department_referral', 'day_of_week'], as_index=False).sum()

# Get the number of patients
daily_temp_patient_pop = dailydepandtime_wait_df[['department_referral', 'patient_waittime', 'day_of_week']].groupby(['department_referral', 'day_of_week'], as_index=False).count()

daily_temp_data = daily_temp_time_data

daily_temp_data['patient_waittime'] = round(daily_temp_time_data['patient_waittime']/daily_temp_patient_pop['patient_waittime'], 0).astype('int')
daily_dep_pivot_df = daily_temp_data.pivot_table(index='department_referral', columns='day_of_week', values='patient_waittime', aggfunc='first')

# Reset column names
daily_dep_pivot_df.columns.name = None

# Reset index name
daily_dep_pivot_df.index.name = 'department'
daily_dep_pivot_df = daily_dep_pivot_df[['Sun','Mon','Tue','Wed','Thu','Fri','Sat']]
daily_dep_pivot_df

,Sun,Mon,Tue,Wed,Thu,Fri,Sat
department,,,,,,,
Cardiology,32,33,39,36,38,38,33
Gastroenterology,33,37,34,38,41,40,29
General Practice,35,36,36,34,35,34,35
Neurology,41,34,36,37,33,37,39
Orthopedics,35,36,35,34,35,35,35
Physiotherapy,33,36,37,41,35,37,35
Renal,39,38,33,34,34,34,33
Unspecified,35,36,35,36,35,35,35


### Insight 11: Hospital Readmission Rates
> 1. First we will need the overall average readmission rate
> 2. Then departmental wise

In [31]:
# The Simplest way to check for readmissions is by checking duplicated patient_ids
readmitted_patient_ids = Data.duplicated(subset=['patient_id'])

# Generate a list of duplicate records
readmitted_patient_records = [Data.iloc[index,:] for index, record in enumerate(readmitted_patient_ids) if record == True]

number_of_readmissions = len(readmitted_patient_records)
number_of_readmissions

0

In [32]:
# Getting the readmission percentage
number_of_admissions = Data['patient_id'].count()
readmission_percentage = number_of_readmissions/number_of_admissions
readmission_percentage

0.0

# SPET 4: VISUALIZING THE DATA

#### Make the main Dataframe pipeline Interactive

In [33]:
iData = Data.interactive()

#### Create a year slider

In [53]:
# Define the panel widgets
year_slider = pn.widgets.IntSlider(name="Year Slider", start=2019, end=2020, value=2020)
year_slider

IntSlider(end=2020, name='Year Slider', start=2019, value=2020)

#### Visualize the overall average waiting time

In [83]:
waiting_time_object = pn.indicators.Number(name="Avg. Waiting time", value=avg_waiting_time.values[0], 
                                           format="{value} Mins", colors=[(20, 'green'), (40, 'gold'), (60, 'red')], 
                                           font_size='30pt',
                                          )
waiting_time_object

Number(colors=[(20, 'green'), ...], font_size='30pt', format='{value} Mins', name='Avg. Waiting time', value=35)

#### Visualize the overall hospital readmission rates

In [84]:
overall_readmission_rate_object = pn.indicators.Number(name="Readmission Rate", value=readmission_percentage, 
                                                       format="{value} %", colors=[(25, 'green'), (40, 'gold'), (50, 'red')], 
                                                      font_size="30pt"
                                                      )
overall_readmission_rate_object

Number(colors=[(25, 'green'), ...], font_size='30pt', format='{value} %', name='Readmission Rate', value=0.0)

In [94]:
overall_number_of_readmissions_object = pn.indicators.Number(name="No. of Readmissions", value=number_of_readmissions, format="{value} Patient(s)", 
                                                             colors=[(0,"orangered")], font_size="22pt")
overall_number_of_readmissions_object

Number(colors=[(0, 'orangered')], font_size='22pt', format='{value} Patient(s)', name='No. of Readmissions', value=0)

#### Visualize the overall average patient satisfaction rating

In [148]:
overall_average_satifaction = pn.widgets.LinearGauge(name="Patient Satisfaction", bounds=(0,10), format="{value:.0f}", horizontal=True,
                                                     colors=[(0.2, 'orangered'), (0.8, 'gold'), (1, 'green')], value=5, width=80, height=300, show_boundaries=True)
overall_average_satifaction

LinearGauge(bounds=(0, 10), colors=[(0.2, 'orangered'), ...], format='{value:.0f}', horizontal=True, name='Patient Satisfaction', show_boundaries=True, value=5, width=80)